In [ ]:
import os
import numpy as np
import pandas as pd
import scanpy as sc
import pybiomart
import seaborn as sns
import matplotlib.pyplot as plt
import loompy as lp
import os, glob
import pickle
import scipy

from arboreto.utils import load_tf_names
from arboreto.algo import grnboost2

from pyscenic.rnkdb import FeatherRankingDatabase as RankingDatabase
from pyscenic.utils import modules_from_adjacencies
from pyscenic.prune import prune2df, df2regulons
from pyscenic.aucell import aucell

import seaborn as sns

import pybiomart
import matplotlib.pyplot as plt
from numpy.random import default_rng
from matplotlib.pyplot import rc_context
sc.settings.verbosity=3         # verbosity: errors (0), warnings (1), info (2), hints (3)
sc.logging.print_header()
    # scanpy==1.7.0
    # anndata==0.7.5
    # umap==0.3.10 numpy==1.18.1
    # scipy==1.4.1 pandas==1.2.2
    # scikit-learn==0.22.2.post1
    # statsmodels==0.12.2
    # python-igraph==0.8.3
    # louvain==0.6.1
    # leidenalg==0.8.3

sc.settings.set_figure_params(dpi=80, facecolor='white')

# read in the counts matrix into a AnnData object
working_dir = './PATH_TO_WORKING_DIR'
output_dir = './PATH_TO_OUTPUT_DIR'
os.chdir(working_dir)

In [ ]:
Patient_ID = 'hcc01'

# reading the data
adata=sc.read_visium('VisiumData/{}'.format(Patient_ID),
    count_file='filtered_feature_bc_matrix.h5', load_images=True)

In [ ]:
# annotate the group of mitochondrial genes as 'mt'
adata.var['mt'] = adata.var_names.str.startswith('MT-')  
sc.pp.calculate_qc_metrics(adata, qc_vars=['mt'], percent_top=None, log1p=False, inplace=True)
adata = adata[adata.obs["pct_counts_mt"] < 40]
print(f"#cells after MT filter: {adata.n_obs}")

sc.pp.filter_genes(adata, min_cells=5)

#save the raw expression matrix gene
raw_expression = adata.X.T

# normalization
sc.pp.normalize_total(adata, inplace=True)

# log transformation
sc.pp.log1p(adata)

# identify highly variable genes
sc.pp.highly_variable_genes(adata, min_mean=0.0125, max_mean=3,
    min_disp=0.5)

sc.pl.highly_variable_genes(adata)

In [ ]:
### manifold embedding and clustering based on transcriptional similarity

# clustering
sc.tl.pca(adata, svd_solver='arpack')
sc.pl.pca_variance_ratio(adata,n_pcs=50,log=True)

sc.pp.pca(adata,n_comps=50)
sc.pp.neighbors(adata,n_pcs=50)
sc.tl.umap(adata)
sc.tl.leiden(adata, key_added='clusters')

#adata.write('VisiumResult/{}/{}_clustered.h5ad'.format(ID, ID), compression='gzip')

# plot covariates to check if there is any particular strucutre in the UMAP
plt.rcParams["figure.figsize"] = (4, 4)


sc.pl.spatial(adata, img_key='hires', color='clusters', size=1.5)

sc.pl.umap(adata, color=["total_counts", "n_genes_by_counts", "clusters"],
    wspace=0.4)


In [ ]:
row_attrs = { 
    "Gene": np.array(adata.var.index) ,
}
col_attrs = { 
    "CellID":  np.array(adata.obs.index) ,
    "nGene": np.array( np.sum(adata.X.transpose()>0 , axis=0)).flatten() ,
    "nUMI": np.array( np.sum(adata.X.transpose() , axis=0)).flatten() ,
}

## create a data folder for the patient
os.chdir(output_dir)
isdir = os.path.isdir('{}/'.format(Patient_ID))
if not isdir:
    os.mkdir('{}/'.format(Patient_ID))

os.chdir('{}/'.format(Patient_ID))

f_loom_path_scenic = '{}_pyscenic.loom'.format(Patient_ID, Patient_ID)
lp.create( f_loom_path_scenic, adata.X.transpose(), row_attrs, col_attrs )

In [ ]:
f_loom_path_scenic  = "{}_pyscenic.loom".format(Patient_ID)
TFS_FNAME = '../../hs_hgnc_tfs.txt'

# path to pyscenic output
f_pyscenic_output = "pyscenic_output.loom"

In [ ]:
!pyscenic grn {f_loom_path_scenic} {TFS_FNAME} -o adj.tsv --num_workers 12

In [ ]:
adjacencies = pd.read_csv("adj.tsv", index_col=False, sep='\t')

In [ ]:
adjacencies.head()

In [ ]:
import glob
# ranking databases
RANKING_DBS_FNAMES = list(map(lambda fn: os.path.join('../../', fn),
                       ['hg38__refseq-r80__10kb_up_and_down_tss.mc9nr.feather',
                       'hg38__refseq-r80__500bp_up_and_100bp_down_tss.mc9nr.feather']))

DBS_PARAM = ' '.join(RANKING_DBS_FNAMES)

# motif databases
f_motif_path = "../../motifs-v9-nr.hgnc-m0.001-o0.0.tbl"

ADJACENCIES_FNAME = "adj.tsv"

DBS_PARAM

In [ ]:
!pyscenic ctx {ADJACENCIES_FNAME} \
    {DBS_PARAM} \
    --annotations_fname {f_motif_path} \
    --expression_mtx_fname {f_loom_path_scenic} \
    --output reg.csv \
    --mask_dropouts \
    --num_workers 12

In [ ]:
!pyscenic aucell \
    {f_loom_path_scenic} \
    reg.csv \
    --output {f_pyscenic_output} \
    --num_workers 12

In [ ]:
pwd

In [ ]:
import json
import zlib
import base64

# collect SCENIC AUCell output
lf = lp.connect( f_pyscenic_output, mode='r+', validate=False )
auc_mtx = pd.DataFrame( lf.ca.RegulonsAUC, index=lf.ca.CellID)
lf.close()

In [ ]:
import umap

# UMAP
runUmap = umap.UMAP(n_neighbors=10, min_dist=0.4, metric='correlation').fit_transform
dr_umap = runUmap( auc_mtx )
pd.DataFrame(dr_umap, columns=['X', 'Y'], index=auc_mtx.index).to_csv( "scenic_umap.txt", sep='\t')


In [ ]:
# scenic output
lf = lp.connect( f_pyscenic_output, mode='r+', validate=False )
meta = json.loads(zlib.decompress(base64.b64decode( lf.attrs.MetaData )))
#exprMat = pd.DataFrame( lf[:,:], index=lf.ra.Gene, columns=lf.ca.CellID)
auc_mtx = pd.DataFrame( lf.ca.RegulonsAUC, index=lf.ca.CellID)
regulons = lf.ra.Regulons
dr_umap = pd.read_csv( 'scenic_umap.txt', sep='\t', header=0, index_col=0 )
###

In [ ]:
auc_mtx.columns = auc_mtx.columns.str.replace('\(','_(')
regulons.dtype.names = tuple( [ x.replace("(","_(") for x in regulons.dtype.names ] )
# regulon thresholds
rt = meta['regulonThresholds']
for i,x in enumerate(rt):
    tmp = x.get('regulon').replace("(","_(")
    x.update( {'regulon': tmp} )

In [ ]:
Embeddings_X = pd.DataFrame( index=lf.ca.CellID )
Embeddings_X = pd.concat( [
        pd.DataFrame(adata.obsm['X_umap'],index=adata.obs.index)[0] ,
        pd.DataFrame(adata.obsm['X_pca'],index=adata.obs.index)[0] ,
        dr_umap['X']
    ], sort=False, axis=1, join='outer' )
Embeddings_X.columns = ['1','2','3']

Embeddings_Y = pd.DataFrame( index=lf.ca.CellID )
Embeddings_Y = pd.concat( [
        pd.DataFrame(adata.obsm['X_umap'],index=adata.obs.index)[1] ,
        pd.DataFrame(adata.obsm['X_pca'],index=adata.obs.index)[1] ,
        dr_umap['Y']
    ], sort=False, axis=1, join='outer' )
Embeddings_Y.columns = ['1','2','3']

In [ ]:
### metadata
metaJson = {}

metaJson['embeddings'] = [
    {
        "id": -1,
        "name": f"Scanpy t-SNE (highly variable genes)"
    },
    {
        "id": 1,
        "name": f"Scanpy UMAP  (highly variable genes)"
    },
    {
        "id": 2,
        "name": "Scanpy PC1/PC2"
    },
    {
        "id": 3,
        "name": "SCENIC AUC UMAP"
    },
]

metaJson["clusterings"] = [{
            "id": 0,
            "group": "Scanpy",
            "name": "Scanpy louvain default resolution",
            "clusters": [],
        }]

metaJson["metrics"] = [
        {
            "name": "nUMI"
        }, {
            "name": "nGene"
        }, {
            "name": "Percent_mito"
        }
]

metaJson["annotations"] = [
    {
        "name": "leiden_clusters_Scanpy",
        "values": list(set( adata.obs['clusters'].astype(np.str) ))
    },
    #{
    #    "name": "Genotype",
    #    "values": list(set(adata.obs['Genotype'].values))
    #},
    #{
    #    "name": "Timepoint",
    #    "values": list(set(adata.obs['Timepoint'].values))
    #},
    #{
    #    "name": "Sample",
    #    "values": list(set(adata.obs['Sample'].values))
    #}
]

# SCENIC regulon thresholds:
metaJson["regulonThresholds"] = rt

for i in range(max(set([int(x) for x in adata.obs['clusters']])) + 1):
    clustDict = {}
    clustDict['id'] = i
    clustDict['description'] = f'Unannotated Cluster {i + 1}'
    metaJson['clusterings'][0]['clusters'].append(clustDict)
    
clusterings = pd.DataFrame()
clusterings["0"] = adata.obs['clusters'].values.astype(np.int64)

In [ ]:
# loom output, generated from a combination of Scanpy and pySCENIC results:
f_final_loom = '{}_scenic_integrated_output.loom'.format(Patient_ID)

def dfToNamedMatrix(df):
    arr_ip = [tuple(i) for i in df.values]
    dtyp = np.dtype(list(zip(df.dtypes.index, df.dtypes)))
    arr = np.array(arr_ip, dtype=dtyp)
    return arr

In [ ]:
col_attrs = {
    "CellID": np.array(adata.obs.index),
    "nUMI": np.array(adata.obs['n_counts'].values),
    "leiden_clusters_Scanpy": np.array( adata.obs['clusters'].values ),
    #"Genotype": np.array(adata.obs['Genotype'].values),
    #"Timepoint": np.array(adata.obs['Timepoint'].values),
    #"Sample": np.array(adata.obs['Sample'].values),
    "Embeddings_X": dfToNamedMatrix(Embeddings_X),
    "Embeddings_Y": dfToNamedMatrix(Embeddings_Y),
    "RegulonsAUC": dfToNamedMatrix(auc_mtx),
    "Clusterings": dfToNamedMatrix(clusterings),
    "ClusterID": np.array(adata.obs['clusters'].values)
}

row_attrs = {
    "Gene": lf.ra.Gene,
    "Regulons": regulons,
}

attrs = {
    "title": "sampleTitle",
    "MetaData": json.dumps(metaJson),
    "Genome": 'hg38',
    "SCopeTreeL1": "",
    "SCopeTreeL2": "",
    "SCopeTreeL3": ""
}

# compress the metadata field:
attrs['MetaData'] = base64.b64encode(zlib.compress(json.dumps(metaJson).encode('ascii'))).decode('ascii')

In [ ]:
auc_mtx
auc_mtx.to_csv('auc_mtx.csv')